# Step01: Data Cleaning
In this notebook, the telecomunications churn data is imported and cleaned.  This data originates from https://www.kaggle.com/becksddf/churn-in-telecoms-dataset

## Import required packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

## Import csv file and create a copy for transformations

In [2]:
churn_og = pd.read_csv('../data/telecoms_churn_raw.csv')
churn = churn_og.copy()

## Replace ' ' with '_' in the column names

In [3]:
churn.columns = churn.columns.str.replace(' ', '_')

In [4]:
churn.head()

,state,account_length,area_code,phone_number,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,...,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,customer_service_calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [5]:
churn['total_day_calls'].value_counts()

102    78
105    75
107    69
95     69
104    68
       ..
149     1
157     1
36      1
30      1
165     1
Name: total_day_calls, Length: 119, dtype: int64

In [6]:
churn['total_day_minutes'].sort_values()

1345      0.0
1397      0.0
2736      2.6
2753      7.8
1986      7.9
        ...  
605     335.5
156     337.4
2594    345.3
985     346.8
365     350.8
Name: total_day_minutes, Length: 3333, dtype: float64

## Replace values in the churn column (target variable)
- Repace True with 1
- Replace False with 0

In [7]:
churn.churn.value_counts()

False    2850
True      483
Name: churn, dtype: int64

In [8]:
churn.churn = churn_og.churn.apply(lambda c: 1 if c == True else 0)

In [9]:
churn.churn.value_counts()

0    2850
1     483
Name: churn, dtype: int64

### Key Takeaway
The True or 1 class, is smaller than the False or 0 class.  Should consider __SMOTE__ function prior to train, test, split


## Review dtypes

In [10]:
churn.dtypes

state                      object
account_length              int64
area_code                   int64
phone_number               object
international_plan         object
voice_mail_plan            object
number_vmail_messages       int64
total_day_minutes         float64
total_day_calls             int64
total_day_charge          float64
total_eve_minutes         float64
total_eve_calls             int64
total_eve_charge          float64
total_night_minutes       float64
total_night_calls           int64
total_night_charge        float64
total_intl_minutes        float64
total_intl_calls            int64
total_intl_charge         float64
customer_service_calls      int64
churn                       int64
dtype: object

## Review each column in turn

### state
- Candidate for categorical variable later on, will create lots of additional columns
- Not all states are represented equally in the data set
    - Max is WV with 106
    - Min is CA with 34

In [11]:
churn.state.describe()

count     3333
unique      51
top         WV
freq       106
Name: state, dtype: object

In [12]:
churn.state.value_counts();

### account_length
- Assuming the value indiates the age of the account

In [13]:
churn.account_length.describe()

count    3333.000000
mean      101.064806
std        39.822106
min         1.000000
25%        74.000000
50%       101.000000
75%       127.000000
max       243.000000
Name: account_length, dtype: float64

### area_code
- Three total values
- 415 represents 1655 rows
- The other two area codes are roughly half
    - 510 represents 840 rows
    - 408 represents 838 rows

In [14]:
churn.area_code.describe()

count    3333.000000
mean      437.182418
std        42.371290
min       408.000000
25%       408.000000
50%       415.000000
75%       510.000000
max       510.000000
Name: area_code, dtype: float64

In [15]:
churn.area_code.value_counts()

415    1655
510     840
408     838
Name: area_code, dtype: int64

### phone_number
- All unique values most likely since the phone number prefix (first three numbers) and line number (last four numbers) are inlcude together and interpereted as a string
- Separating prefix and line numbers to investigate any categorical relationships yields:
    - Phone prefix values total 96 discrete values
        - Will keep for the time being, may eventually drop this column
    - Phone line numbers are completly unique
        - These provide no value in that they are completely unique, dropping from the dataframe

In [16]:
churn.phone_number.describe()

count         3333
unique        3333
top       399-7268
freq             1
Name: phone_number, dtype: object

In [17]:
churn.phone_number

0       382-4657
1       371-7191
2       358-1921
3       375-9999
4       330-6626
          ...   
3328    414-4276
3329    370-3271
3330    328-8230
3331    364-6381
3332    400-4344
Name: phone_number, Length: 3333, dtype: object

In [18]:
churn_og['phone number'].apply(lambda p: p[0:3]).describe()

### Could be interesting to include, will add to the dataframe

count     3333
unique      96
top        405
freq        53
Name: phone number, dtype: object

In [19]:
### Adding phone_prefix to dataframe
churn['phone_prefix'] = churn_og['phone number'].apply(lambda p: p[0:3])

In [20]:
churn_og['phone number'].apply(lambda p: p[4:]).describe()

### Adds no value and will not be added to the dataframe

count     3333
unique    3333
top       5699
freq         1
Name: phone number, dtype: object

In [21]:
churn = churn.drop(columns = ['phone_number'])

### international_plan
- Binary yes/no
- Candidate for categorical variable
- Far more no values (3010) then yes values (323)

In [22]:
churn.international_plan.describe()

count     3333
unique       2
top         no
freq      3010
Name: international_plan, dtype: object

In [23]:
churn.international_plan.value_counts()

no     3010
yes     323
Name: international_plan, dtype: int64

### voice_mail_plan
- Binary yes/no
- More no values (2411) than yes values (922)

In [24]:
churn.voice_mail_plan.describe()

count     3333
unique       2
top         no
freq      2411
Name: voice_mail_plan, dtype: object

In [25]:
churn.voice_mail_plan.value_counts()

no     2411
yes     922
Name: voice_mail_plan, dtype: int64

### number_vmail_messages
- Integers ranging from zero to 51 voicemails
- Candidate for binning these values

In [26]:
churn.number_vmail_messages.describe()

count    3333.000000
mean        8.099010
std        13.688365
min         0.000000
25%         0.000000
50%         0.000000
75%        20.000000
max        51.000000
Name: number_vmail_messages, dtype: float64

In [27]:
churn.number_vmail_messages.min(), churn.number_vmail_messages.max()

(0, 51)

In [50]:
churn.number_vmail_messages.value_counts()

0     2411
31      60
29      53
28      51
33      46
27      44
30      44
24      42
26      41
32      41
25      37
23      36
36      34
35      32
22      32
39      30
37      29
34      29
21      28
38      25
20      22
19      19
40      16
42      15
17      14
41      13
16      13
43       9
15       9
18       7
44       7
14       7
45       6
12       6
46       4
13       4
47       3
8        2
48       2
50       2
9        2
11       2
49       1
10       1
4        1
51       1
Name: number_vmail_messages, dtype: int64

### day metrics (total_day_minutes, total_day_calls, total_day_charge)
- Candidates for binning

In [29]:
day = churn[['total_day_minutes', 'total_day_calls', 'total_day_charge']]

In [30]:
day.dtypes

total_day_minutes    float64
total_day_calls        int64
total_day_charge     float64
dtype: object

In [31]:
day.describe()

,total_day_minutes,total_day_calls,total_day_charge
count,3333.000000,3333.000000,3333.000000
mean,179.775098,100.435644,30.562307
std,54.467389,20.069084,9.259435
min,0.000000,0.000000,0.000000
25%,143.700000,87.000000,24.430000
50%,179.400000,101.000000,30.500000
75%,216.400000,114.000000,36.790000
max,350.800000,165.000000,59.640000


### eve metrics (total_eve_minutes, total_eve_calls, total_eve_charge)

In [32]:
eve = churn[['total_eve_minutes', 'total_eve_calls', 'total_eve_charge']]

In [33]:
eve.dtypes

total_eve_minutes    float64
total_eve_calls        int64
total_eve_charge     float64
dtype: object

In [34]:
eve.describe()

,total_eve_minutes,total_eve_calls,total_eve_charge
count,3333.000000,3333.000000,3333.000000
mean,200.980348,100.114311,17.083540
std,50.713844,19.922625,4.310668
min,0.000000,0.000000,0.000000
25%,166.600000,87.000000,14.160000
50%,201.400000,100.000000,17.120000
75%,235.300000,114.000000,20.000000
max,363.700000,170.000000,30.910000


### night metrics (total_night_minutes, total_night_calls, total_eve_charge)

In [35]:
night = churn[['total_night_minutes', 'total_night_calls', 'total_night_charge']]

In [36]:
night.dtypes

total_night_minutes    float64
total_night_calls        int64
total_night_charge     float64
dtype: object

In [37]:
night.describe()

,total_night_minutes,total_night_calls,total_night_charge
count,3333.000000,3333.000000,3333.000000
mean,200.872037,100.107711,9.039325
std,50.573847,19.568609,2.275873
min,23.200000,33.000000,1.040000
25%,167.000000,87.000000,7.520000
50%,201.200000,100.000000,9.050000
75%,235.300000,113.000000,10.590000
max,395.000000,175.000000,17.770000


### intl metrics (total_intl_minutes, total_intl_calls, total_intl_charge)

In [38]:
intl = churn[['total_intl_minutes', 'total_intl_calls', 'total_intl_charge']]

In [39]:
intl.dtypes

total_intl_minutes    float64
total_intl_calls        int64
total_intl_charge     float64
dtype: object

In [40]:
intl.describe()

,total_intl_minutes,total_intl_calls,total_intl_charge
count,3333.000000,3333.000000,3333.000000
mean,10.237294,4.479448,2.764581
std,2.791840,2.461214,0.753773
min,0.000000,0.000000,0.000000
25%,8.500000,3.000000,2.300000
50%,10.300000,4.000000,2.780000
75%,12.100000,6.000000,3.270000
max,20.000000,20.000000,5.400000


### customer_service_calls
- Most common vlaue is 1 with 1,181 instances
- Only 13 greater than or equal to 7

In [41]:
churn.customer_service_calls.describe()

count    3333.000000
mean        1.562856
std         1.315491
min         0.000000
25%         1.000000
50%         1.000000
75%         2.000000
max         9.000000
Name: customer_service_calls, dtype: float64

In [42]:
churn.customer_service_calls.value_counts()

1    1181
2     759
0     697
3     429
4     166
5      66
6      22
7       9
9       2
8       2
Name: customer_service_calls, dtype: int64

In [43]:
churn['total_calls'] = churn['total_day_calls'] + churn['total_eve_calls'] + churn['total_night_calls'] + churn['total_intl_calls']

In [44]:
churn['total_calls'].describe()

count    3333.000000
mean      305.137114
std        34.448164
min       191.000000
25%       282.000000
50%       305.000000
75%       328.000000
max       416.000000
Name: total_calls, dtype: float64

In [45]:
churn['total_minutes'] = churn['total_day_minutes'] + churn['total_eve_minutes'] + churn['total_night_minutes']+ churn['total_intl_minutes']

In [46]:
churn['total_minutes'].describe()

count    3333.000000
mean      591.864776
std        89.954251
min       284.300000
25%       531.500000
50%       593.600000
75%       652.400000
max       885.000000
Name: total_minutes, dtype: float64

In [47]:
churn['total_charge'] = churn['total_day_charge']+ churn['total_eve_charge']+ churn['total_night_charge']+ churn['total_intl_charge']

In [48]:
churn

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,...,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,customer_service_calls,churn,phone_prefix,total_calls,total_minutes,total_charge
0,KS,128,415,no,yes,25,265.1,110,45.07,197.4,...,11.01,10.0,3,2.70,1,0,382,303,717.2,75.56
1,OH,107,415,no,yes,26,161.6,123,27.47,195.5,...,11.45,13.7,3,3.70,1,0,371,332,625.2,59.24
2,NJ,137,415,no,no,0,243.4,114,41.38,121.2,...,7.32,12.2,5,3.29,0,0,358,333,539.4,62.29
3,OH,84,408,yes,no,0,299.4,71,50.90,61.9,...,8.86,6.6,7,1.78,2,0,375,255,564.8,66.80
4,OK,75,415,yes,no,0,166.7,113,28.34,148.3,...,8.41,10.1,3,2.73,3,0,330,359,512.0,52.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,AZ,192,415,no,yes,36,156.2,77,26.55,215.5,...,12.56,9.9,6,2.67,2,0,414,292,660.7,60.10
3329,WV,68,415,no,no,0,231.1,57,39.29,153.4,...,8.61,9.6,4,2.59,3,0,370,239,585.4,63.53
3330,RI,28,510,no,no,0,180.8,109,30.74,288.8,...,8.64,14.1,6,3.81,2,0,328,264,675.6,67.74
3331,CT,184,510,yes,no,0,213.8,105,36.35,159.6,...,6.26,5.0,10,1.35,2,0,364,336,517.6,57.53


## Export data to csv

In [49]:
churn.to_csv('../data/vd_churn_cleaned.csv')